In [47]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.ticker as ticker
from matplotlib.cm import ScalarMappable
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import pickle

from math import radians, cos, sin, sqrt, atan2

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import d2_absolute_error_score
from sklearn.metrics import mean_squared_error
# from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.cluster import HDBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import Normalize
from matplotlib.colors import ListedColormap

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import random

import libpysal as lp
from libpysal.weights import DistanceBand
from esda.moran import Moran

# from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.basemap import Basemap

# For sanity checking
from sklearn.decomposition import PCA

from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay

from scipy.stats import norm
from scipy.stats import kurtosis
from scipy.stats import gaussian_kde
from scipy.spatial.distance import cdist

import glob

from geopy.distance import geodesic

from joblib import dump, load
from tqdm.notebook import tqdm

import os
import shutil
import pysplit

# These packages are used when the code to plot the decision tree is uncommented
# See **TREE PLOTTING** section and un-uncomment the code there
from sklearn.tree import plot_tree
from sklearn.tree import export_text

import warnings
from geopy.distance import geodesic

# import umap
from geopy.distance import great_circle
from joblib import Parallel, delayed
# from haversine import haversine, Unit
# from numba import jit
from functools import reduce

# plt.ioff()

In [48]:
# Define a function that calcualte error metrics from predicted and actual values
def reg_model_metrics(actual,pred):
    MSE = mean_squared_error(actual,pred)
    RMSE = np.sqrt(MSE)
    actual_mean = np.mean(actual)
    RRMSE = 100*RMSE/actual_mean
    MAE = mean_absolute_error(actual, pred)
    R2 = r2_score(actual,pred)
    D2 = d2_absolute_error_score(actual, pred)
    MAXErr = max_error(actual, pred)
    EVS = explained_variance_score(actual, pred)
    return MSE, RMSE, RRMSE, MAE, R2, D2, MAXErr, EVS 

def calculate_and_organize_metrics_model(actual, pred, model, ats):
     
    # Calculate standard error metrics
    mse, rmse, rrmse, mae, r2, d2, max_err, evs = reg_model_metrics(actual, pred)
    
    # Organize metrics into a dictionary
    metrics_dict = {
        "Model": model,
        "ATS" : ats,
        f"MSE": mse,
        f"RMSE": rmse,
        f"RRMSE": rrmse,
        f"MAE": mae,
        f"R2": r2,
        f"D2": d2,
        f"MaxErr": max_err,
        f"EVS": evs,
    }
    
    # Convert dictionary to DataFrame
    metrics_df = pd.DataFrame(metrics_dict, index=[0])
   
    return metrics_df

def scatter_plot(actual, pred, title, var_of_int, save_path):
    # Color Matching for clarity
    if var_of_int == 'CH4':
        color = 'blue'
        cm = 'Blues'
    elif var_of_int == 'DMS':
        color = 'purple'
        cm = 'Purples'
    elif var_of_int == 'CO':
        color = 'cyan'
        cm = 'YlGnBu'
    elif var_of_int == 'O3':
        color = 'gold'
        cm = 'YlOrBr'
    elif var_of_int == 'Benzene':
        color = 'red'
        cm = 'Reds'
    elif var_of_int == 'CH3Br':
        color = 'mediumseagreen'
        cm = 'Greens'
    elif var_of_int == 'Ethane':
        color = 'orange'
        cm = 'Oranges'
    else:
        color = 'gray'
        cm = 'Greys'
        
    # if comp_flag:
    #     cm = "Greys"
    
    MSE, RMSE, RRMSE, MAE, R2, D2, MAXErr, EVS = reg_model_metrics(actual, pred)
        
    fig,ax = plt.subplots(figsize=(8, 6))
    ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)
    ax.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', lw=2)
    # text = r"R2 = %.2f" % (R2); text += "\n";
    # text += r"D2 = %.2f" % (D2); text += "\n";
    text = r"MAE = %.2f" % (MAE); text += "\n";
    text += r"RMSE = %.2f" % (RMSE); text += "\n";
    text += r"MSE = %.2f" % (MSE);     
    plt.annotate(text, xy=(0.01, 0.9), xycoords='axes fraction',color='black', fontsize=10,bbox=dict(facecolor='none', edgecolor='none'))
    ax.set_xlabel(f'Measured {var_of_int}')
    ax.set_ylabel(f'Predicted {var_of_int}')
    # Set log scale for x and y axes
    ax.set_xscale('log')
    ax.set_yscale('log')
    # if var_of_int != 'DMS':
    # Create locators to specify the ticks on both axes
    ax.xaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=10))  # Controls major ticks
    ax.xaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs="auto"))  # Controls minor ticks

    ax.yaxis.set_major_locator(ticker.LogLocator(base=10.0, numticks=10))
    ax.yaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs="auto"))

    # Ensure minor gridlines are visible and major gridlines are configured
    ax.grid(visible=True, which="major", linestyle="--", linewidth=0.5)  # Major gridlines
    ax.grid(visible=True, which="minor", linestyle=":", linewidth=0.5)  # Minor gridlines
    ax.set_title(title)
    # plt.grid()
    plt.savefig(save_path)
    # plt.show()
    plt.clf()
    return None

In [49]:
df = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\df_preprocessed.csv")
atom_gmi = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\atom_gmi.csv")
atom_cesm = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\atom_cesm.csv")
seac4rs = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\seac4rs_geos-chem.csv")
winter = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\winter_geos-chem.csv")
korusaq = pd.read_csv(r"C:\Users\vwgei\Documents\PVOCAL\data\V4\joined_model_data\korusaq_micp.csv")

In [50]:
atom_gmi_df = pd.merge(atom_gmi, df, how='inner', on='iindex', suffixes=("_gmi", "_df"))
atom_cesm_df = pd.merge(atom_cesm, df, how='inner', on='iindex', suffixes=("_cesm", "_df"))
seac4rs_df = pd.merge(seac4rs, df, how='inner', on='iindex', suffixes=("_seac4rs", "_df"))
winter_df = pd.merge(winter, df, how='inner', on='iindex', suffixes=("_winter", "_df"))
korusaq_df = pd.merge(korusaq, df, how='inner', on='iindex', suffixes=("_korusaq", "_df"))

In [51]:
with open(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\global_models\V4_CO\data\CO_global_X_test.pkl", 'rb') as file:
    co_test = pickle.load(file)
co_pred_test = np.load(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\PASTEL_combined\V4_CO\CO_PASTEL_combined_pred_test.npy")

pastel_co = pd.DataFrame({
    "iindex": co_test.index,
    "PASTEL_CO": co_pred_test
})


with open(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\global_models\V4_CH4\data\CH4_global_X_test.pkl", 'rb') as file:
    ch4_test = pickle.load(file)
ch4_pred_test = np.load(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\PASTEL_combined\V4_CH4\CH4_PASTEL_combined_pred_test.npy")

pastel_ch4 = pd.DataFrame({
    "iindex": ch4_test.index,
    "PASTEL_CH4": ch4_pred_test
})


with open(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\global_models\V4_CH3Br\data\CH3Br_global_X_test.pkl", 'rb') as file:
    ch3br_test = pickle.load(file)
ch3br_pred_test = np.load(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\PASTEL_combined\V4_CH3Br\CH3Br_PASTEL_combined_pred_test.npy")

pastel_ch3br = pd.DataFrame({
    "iindex": ch3br_test.index,
    "PASTEL_CH3Br": ch3br_pred_test
})


with open(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\global_models\V4_DMS\data\DMS_global_X_test.pkl", 'rb') as file:
    dms_test = pickle.load(file)
dms_pred_test = np.load(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\PASTEL_combined\V4_DMS\DMS_PASTEL_combined_pred_test.npy")

pastel_dms = pd.DataFrame({
    "iindex": dms_test.index,
    "PASTEL_DMS": dms_pred_test
})


with open(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\global_models\V4_Ethane\data\Ethane_global_X_test.pkl", 'rb') as file:
    ethane_test = pickle.load(file)
ethane_pred_test = np.load(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\PASTEL_combined\V4_Ethane\Ethane_PASTEL_combined_pred_test.npy")

pastel_ethane = pd.DataFrame({
    "iindex": ethane_test.index,
    "PASTEL_Ethane": ethane_pred_test
})


with open(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\global_models\V4_O3\data\O3_global_X_test.pkl", 'rb') as file:
    o3_test = pickle.load(file)
o3_pred_test = np.load(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\PASTEL_combined\V4_O3\O3_PASTEL_combined_pred_test.npy")

pastel_o3 = pd.DataFrame({
    "iindex": o3_test.index,
    "PASTEL_O3": o3_pred_test
})

In [52]:
# List of DataFrames to merge
dfs = [pastel_co, pastel_ch4, pastel_ethane, pastel_dms, pastel_ch3br, pastel_o3]

# Merge all DataFrames on 'iindex' using outer join
pastel = reduce(lambda left, right: pd.merge(left, right, how="outer", on="iindex"), dfs)

In [53]:
pastel

,iindex,PASTEL_CO,PASTEL_CH4,PASTEL_Ethane,PASTEL_DMS,PASTEL_CH3Br,PASTEL_O3
0,21026,230.802942,NaN,NaN,NaN,NaN,NaN
1,7283,116.191180,NaN,NaN,NaN,NaN,NaN
2,612,60.258867,NaN,NaN,NaN,NaN,NaN
3,33869,387.163510,NaN,NaN,12.99987,NaN,NaN
4,38732,57.658160,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
29876,35394,NaN,NaN,NaN,NaN,NaN,50.291097
29877,35827,NaN,NaN,NaN,NaN,NaN,55.950217
29878,25088,NaN,NaN,NaN,NaN,NaN,51.205646
29879,22539,NaN,NaN,NaN,NaN,NaN,46.140839


In [54]:
atom_gmi_f = pd.merge(atom_gmi_df, pastel, how='inner', on='iindex')
atom_cesm_f = pd.merge(atom_cesm_df, pastel, how='inner', on='iindex')
seac4rs_f = pd.merge(seac4rs_df, pastel, how='inner', on='iindex')
winter_f = pd.merge(winter_df, pastel, how='inner', on='iindex')
korusaq_f = pd.merge(korusaq_df, pastel, how='inner', on='iindex')

final_dfs = [atom_gmi_f,atom_cesm_f,seac4rs_f,winter_f,korusaq_f]

for df in final_dfs:
    # Replace WAS nodata value with np.nan for consistency
    df.replace(-999999.0, np.nan, inplace=True)
    df.replace(-888888.0, np.nan, inplace=True)
    df.replace(-888888888.0, np.nan, inplace=True)
    df.replace(-999999999.0, np.nan, inplace=True)
    df.replace(-99999.000000, np.nan, inplace=True)
    df.replace(-999.990000, np.nan, inplace=True)
    df.replace(-9.999999e+09, np.nan, inplace=True)
    df.replace(-9.999990e+08, np.nan, inplace=True)
    df.replace(-8.888888e+06, np.nan, inplace=True)
    df.replace(-9.999999e+06, np.nan, inplace=True)

    # Replace WAS nodata value with np.nan for consistency
    df.replace(-8888, np.nan, inplace=True)
    df.replace(-999, np.nan, inplace=True)
    df.replace(-888, np.nan, inplace=True)
    df.replace(-777, np.nan, inplace=True)
    df.replace(-777.770000, np.nan, inplace=True)
    df.replace(-888.800000, np.nan, inplace=True)
    df.replace(-888.880000, np.nan, inplace=True)
    df.replace(10000000, np.nan, inplace=True)

In [55]:
atom_gmi_f["GMI_Ethane_GMI"] = atom_gmi_f["GMI_Ethane_GMI"] * 1e3

In [56]:
atom_gmi_f["GMI_CH3Br_GMI"] = atom_gmi_f["GMI_CH3Br_GMI"] * 1e3

In [57]:
ATS_names = ["CO", "CH4", "CH3Br", "DMS", "O3", "Ethane"]

# Create a mapping of names to specific column names for the GMI and PASTEL models
model_mapping = {
    "CO": ("CO_df", "GMI_CO_GMI", "PASTEL_CO"),
    "CH3Br": ("CH3Br", "GMI_CH3Br_GMI", "PASTEL_CH3Br"),
    "CH4": ("CH4_df", "GMI_CH4_GMI", "PASTEL_CH4"),
    "Ethane": ("Ethane", "GMI_Ethane_GMI", "PASTEL_Ethane"),
    "O3": ("O3_df", "GMI_O3_GMI", "PASTEL_O3"),
}

# Initialize an empty list to store results
metrics_list = []

# Loop through species and calculate metrics
for species, (data_col, gmi_col, pastel_col) in model_mapping.items():
    # Drop NaN values from both data and target columns
    merged_gmi = pd.concat([atom_gmi_f[data_col], atom_gmi_f[gmi_col]], axis=1).dropna()
    merged_pastel = pd.concat([atom_gmi_f[data_col], atom_gmi_f[pastel_col]], axis=1).dropna()
    
    # Ensure the same rows are kept in both DataFrames (i.e., intersection of indexes)
    merged_gmi = merged_gmi.loc[merged_gmi.index.intersection(merged_pastel.index)]
    
    # scatter_plot(merged_gmi[data_col], merged_gmi[gmi_col], "gmi_scatter", species)
    # scatter_plot(merged_pastel[data_col], merged_pastel[pastel_col], "pastel_sctter", species)

    # Calculate metrics for GMI and PASTEL after dropping NaN
    metrics_list.append(
        calculate_and_organize_metrics_model(merged_gmi[data_col], merged_gmi[gmi_col], "GMI", species)
    )
    metrics_list.append(
        calculate_and_organize_metrics_model(merged_pastel[data_col], merged_pastel[pastel_col], "PASTEL", species)
    )

# Combine all metrics into a single DataFrame
model_metrics = pd.concat(metrics_list, ignore_index=True)

model_metrics[['Model', 'ATS', 'RMSE', 'MAE', "R2"]]

# ATS_names = ["CO", "CH4", "CH3Br", "DMS", "O3", "Ethane"]

# gmi_co = calculate_and_organize_metrics_model(atom_gmi_f['CO_df'], atom_gmi_f['GMI_CO_GMI'], "GMI", "CO")
# gmi_ch3br = calculate_and_organize_metrics_model(atom_gmi_f['CH3Br'], atom_gmi_f['GMI_CH3Br_GMI'], "GMI", "CH3Br")
# gmi_ch4 = calculate_and_organize_metrics_model(atom_gmi_f['CH4_df'], atom_gmi_f['GMI_CH4_GMI'], "GMI", "CH4")
# gmi_ethane = calculate_and_organize_metrics_model(atom_gmi_f['Ethane'], atom_gmi_f['GMI_Ethane_GMI'], "GMI", "C2H6")
# gmi_o3 = calculate_and_organize_metrics_model(atom_gmi_f['O3'], atom_gmi_f['GMI_O3_GMI'], "GMI", "O3")
# pastel_co = calculate_and_organize_metrics_model(atom_gmi_f['CO_df'], atom_gmi_f['PASTEL_CO'], "PASTEL", "CO")
# pastel_ch3br = calculate_and_organize_metrics_model(atom_gmi_f['CH3Br'], atom_gmi_f['PASTEL_CH3Br'], "PASTEL", "CH3Br")
# pastel_ch4 = calculate_and_organize_metrics_model(atom_gmi_f['CH4_df'], atom_gmi_f['PASTEL_CH4'], "PASTEL", "CH4")
# pastel_ethane = calculate_and_organize_metrics_model(atom_gmi_f['Ethane'], atom_gmi_f['PASTEL_Ethane'], "PASTEL", "C2H6")
# pastel_o3 = calculate_and_organize_metrics_model(atom_gmi_f['O3'], atom_gmi_f['PASTEL_O3'], "PASTEL", "O3")


# model_metrics = pd.concat([gmi_co, pastel_co, 
#                            gmi_ch3br, pastel_ch3br, 
#                            gmi_ch4, pastel_ch4, 
#                            gmi_ethane, pastel_ethane,
#                            gmi_o3, pastel_o3], ignore_index=True)

,Model,ATS,RMSE,MAE,R2
0,GMI,CO,20.121550,15.084427,0.576432
1,PASTEL,CO,11.530789,7.110605,0.860903
2,GMI,CH3Br,5.171760,5.126703,-72.106628
3,PASTEL,CH3Br,0.387029,0.269500,0.590582
4,GMI,CH4,26.528310,21.700120,0.742828
5,PASTEL,CH4,13.855175,8.643548,0.929850
6,GMI,Ethane,399.624985,180.277357,0.572089
7,PASTEL,Ethane,230.985029,109.462410,0.857040
8,GMI,O3,28.101071,12.733067,0.901025
9,PASTEL,O3,28.361160,11.933836,0.899185


In [58]:
for c in atom_cesm_f.columns:
    print(c)

iindex
Datetime_cesm
MMS_P
MMS_T
MMS_TAS
U
V
W
MMS_TEDR
MMS_REYN
LAT_cesm
LON_cesm
ALT_cesm
MMS_POT
MMS_ROLL
MMS_HDG
MMS_PITCH
MMS_YAW
MMS_AOA
DLH_H2O_H2O_DLH
DLH_H2O_RHi_DLH
DLH_H2O_RHw_DLH
ATHOS_OHI_OHInterference_ATHOS
OH
ATHOS_HOx_HO2_ATHOS
ATHOS_OHR_T_OHR
ATHOS_OHR_P_OHR
ATHOS_OHR_OHReactivity_OHR
Influences_Julian_Day
Influences_Lat
Influences_Lon
Influences_Pres
Influences_Prob_ConvInf
Influences_Prob_ConvInf_Land
Influences_ConvInf_CldTop
Influences_Days_Since_ConvInf
Influences_Days_Since_Most_Recent_ConvInf
Influences_Prob_BdyInf
Influences_Days_Since_BdyInf_Most_Recent
Influences_Days_Since_BdyInf_All
Influences_Prob_Land_BdyInf
Influences_Days_Since_Land_BdyInf_Most_Recent
Influences_Prob_SeaIceInf
Influences_Prob_StratInf
Influences_Days_Since_StratInf_Most_Recent
Influences_Days_Since_StratInf_All
Influences_Prob_FireInf
Influences_Days_Since_FireInf_Most_Recent
Influences_Days_Since_FireInf_All
Influences_Sunlight_Exposure
CO2_cesm
CH4_cesm
CO_cesm
GMI_lat
GMI_lon
GMI_St

In [59]:
atom_cesm_f["O3_df"]

0       48.920000
1       74.342857
2       71.885714
3       99.014286
4       95.383333
          ...    
4604    52.540000
4605          NaN
4606          NaN
4607          NaN
4608          NaN
Name: O3_df, Length: 4609, dtype: float64

In [60]:
atom_cesm_f["CH4_CESM"] = atom_cesm_f["CH4_CESM"] * 1e9
atom_cesm_f["O3_CESM"] = atom_cesm_f["O3_CESM"] * 1e9
atom_cesm_f["CO_CESM"] = atom_cesm_f["CO_CESM"] * 1e9
atom_cesm_f["CH3BR_CESM"] = atom_cesm_f["CH3BR_CESM"] * 1e12
atom_cesm_f["C2H6_CESM"] = atom_cesm_f["C2H6_CESM"] * 1e12
atom_cesm_f["DMS_CESM"] = atom_cesm_f["DMS_CESM"] * 1e12

In [61]:
ATS_names = ["CO", "CH4", "CH3Br", "DMS", "O3", "Ethane"]

# Create a mapping of names to specific column names for the GMI and PASTEL models
model_mapping = {
    "CO": ("CO_df", "CO_CESM", "PASTEL_CO"),
    "CH3Br": ("CH3Br_df", "CH3BR_CESM", "PASTEL_CH3Br"),
    "CH4": ("CH4_df", "CH4_CESM", "PASTEL_CH4"),
    "Ethane": ("Ethane_df", "C2H6_CESM", "PASTEL_Ethane"),
    "O3": ("O3_df", "O3_CESM", "PASTEL_O3"),
    "DMS" : ("DMS_df", "DMS_CESM", "PASTEL_DMS"),
}

# Initialize an empty list to store results
metrics_list = []

# Loop through species and calculate metrics
for species, (data_col, cesm_col, pastel_col) in model_mapping.items():
    # Drop NaN values from both data and target columns
    merged_cesm = pd.concat([atom_cesm_f[data_col], atom_cesm_f[cesm_col]], axis=1).dropna()

    if species == 'DMS':
        merged_cesm = merged_cesm[merged_cesm[cesm_col] >= 1]

    merged_pastel = pd.concat([atom_cesm_f[data_col], atom_cesm_f[pastel_col]], axis=1).dropna()
    
    # Ensure the same rows are kept in both DataFrames (i.e., intersection of indexes)
    merged_cesm = merged_cesm.loc[merged_cesm.index.intersection(merged_pastel.index)]
    
    # print(len(merged_cesm))

    # scatter_plot(merged_cesm[data_col], merged_cesm[cesm_col], "cesm_scatter", species)
    # scatter_plot(merged_pastel[data_col], merged_pastel[pastel_col], "pastel_sctter", species)

    # Calculate metrics for GMI and PASTEL after dropping NaN
    metrics_list.append(
        calculate_and_organize_metrics_model(merged_cesm[data_col], merged_cesm[cesm_col], "CESM", species)
    )
    metrics_list.append(
        calculate_and_organize_metrics_model(merged_pastel[data_col], merged_pastel[pastel_col], "PASTEL", species)
    )

# Combine all metrics into a single DataFrame
model_metrics = pd.concat(metrics_list, ignore_index=True)

model_metrics[['Model', 'ATS', 'RMSE', 'MAE', "R2"]]

,Model,ATS,RMSE,MAE,R2
0,CESM,CO,28.355410,21.444255,0.158853
1,PASTEL,CO,11.530789,7.110605,0.860903
2,CESM,CH3Br,0.802643,0.637930,-0.760859
3,PASTEL,CH3Br,0.387029,0.269500,0.590582
4,CESM,CH4,20.901074,15.719690,0.840360
5,PASTEL,CH4,13.855175,8.643548,0.929850
6,CESM,Ethane,639.833650,381.726601,-0.096939
7,PASTEL,Ethane,230.985029,109.462410,0.857040
8,CESM,O3,39.830331,21.373700,0.801159
9,PASTEL,O3,28.361160,11.933836,0.899185


In [62]:
for c in seac4rs_f.columns:
    print(c)

iindex
Fractional_Day
STARTTIME
STOPTIME
UTC
JDAY
INDEX
FLIGHT
LOCAL_SUN_TIME
LAT_seac4rs
LON_seac4rs
ALT_seac4rs
PRESSURE_obs
TEMPERATURE
THETA
O3COLUMN
SZA
WNS
WND
GPS_ALT
RadarAlt
GRD_SPD
TAS
IAS
MachNumber
VerticalSpeed
HDG
TRK
DriftAngle
PITCH
ROLL
StaticTemp_Experimenter
PotentialTemp
Dewpoint
TotalTemp
TotalTemp_Experimenter
IR_SurfTemp
CabinPressure
SolarZenithAngle
AircraftSunElevation
SunAzimuth
AircraftSunAzimuth
H2O_MixingRatio
H2O_VaporPres
H2O_SatVaporPresWater
H2O_SatVaporPresIce
H2O_RelativeHumidity
StaticPressure_MMS
StaticTemp_MMS
TAS_MMS
U
V
W
TEDR
REYN
GPS_LAT_MMS
GPS_LON_MMS
GPS_ALT_MMS
THETA_MMS
ROLL_MMS
HDG_MMS
PITCH_MMS
U_AIMMS20
V_AIMMS20
W_AIMMS20
CO2_seac4rs
CO_DACOM
CO-SourceFlag_DACOM
H2O_DLH
CH2O_CAMS
CH2O_LIF
NO2_ESRL
NOy_ESRL
NO_ESRL
O3_obs
NO2_TDLIF
MPN_TDLIF
PNs_TDLIF
ANs_TDLIF
HNO3_NO3-lt1um_SAGA
Sulfate-lt1um_SAGA
Cl_SAGA-AERO
Br_SAGA-AERO
NO3_SAGA-AERO
SO4_SAGA-AERO
C2O4_SAGA-AERO
Na_SAGA-AERO
NH4_SAGA-AERO
K_SAGA-AERO
Mg_SAGA-AERO
Ca_SAGA-AERO
PAN_

In [63]:
seac4rs_f["CH3Br_seac4rs"] = seac4rs_f["CH3Br_seac4rs"] * 1e3
seac4rs_f["C2H6"] = seac4rs_f["C2H6"] * 1e3
seac4rs_f["DMS_seac4rs"] = seac4rs_f["DMS_seac4rs"] * 1e3

In [64]:
ATS_names = ["CO", "CH4", "CH3Br", "DMS", "O3", "Ethane"]

# Create a mapping of names to specific column names for the GMI and PASTEL models
model_mapping = {
    # "CO": ("CO_obs", "CO_geos-chem", "PASTEL_CO"),
    "CH3Br": ("CH3Br_df", "CH3Br_seac4rs", "PASTEL_CH3Br"),
    "Ethane": ("Ethane", "C2H6", "PASTEL_Ethane"),
    "O3": ("O3", "O3_geos-chem", "PASTEL_O3"),
    "DMS" : ("DMS_df", "DMS_seac4rs", "PASTEL_DMS"),
}

# Initialize an empty list to store results
metrics_list = []

# Loop through species and calculate metrics
for species, (data_col, seac4rs_col, pastel_col) in model_mapping.items():
    # Drop NaN values from both data and target columns
    merged_seac4rs = pd.concat([seac4rs_f[data_col], seac4rs_f[seac4rs_col]], axis=1).dropna()

    if species == 'DMS':
        merged_seac4rs = merged_seac4rs[merged_seac4rs[seac4rs_col] >= 1]

    merged_pastel = pd.concat([seac4rs_f[data_col], seac4rs_f[pastel_col]], axis=1).dropna()
    
    # Ensure the same rows are kept in both DataFrames (i.e., intersection of indexes)
    merged_seac4rs = merged_seac4rs.loc[merged_seac4rs.index.intersection(merged_pastel.index)]
    
    print(len(merged_seac4rs))

    # scatter_plot(merged_seac4rs[data_col], merged_seac4rs[seac4rs_col], "seac4rs_scatter", species)
    # scatter_plot(merged_pastel[data_col], merged_pastel[pastel_col], "pastel_sctter", species)

    # Calculate metrics for GMI and PASTEL after dropping NaN
    metrics_list.append(
        calculate_and_organize_metrics_model(merged_seac4rs[data_col], merged_seac4rs[seac4rs_col], "GEOS-Chem", species)
    )
    metrics_list.append(
        calculate_and_organize_metrics_model(merged_pastel[data_col], merged_pastel[pastel_col], "PASTEL", species)
    )

# Combine all metrics into a single DataFrame
model_metrics = pd.concat(metrics_list, ignore_index=True)

model_metrics[['Model', 'ATS', 'RMSE', 'MAE', "R2"]]

473
465
467
11


,Model,ATS,RMSE,MAE,R2
0,GEOS-Chem,CH3Br,3.630744,1.149814,0.005336
1,PASTEL,CH3Br,3.786270,1.408916,-0.081703
2,GEOS-Chem,Ethane,1401.215634,896.087849,-0.475390
3,PASTEL,Ethane,1384.939133,959.444542,-0.441313
4,GEOS-Chem,O3,17.830626,13.588456,0.215829
5,PASTEL,O3,30.693972,22.835491,-1.323721
6,GEOS-Chem,DMS,18.092392,14.162455,-0.102566
7,PASTEL,DMS,15.710799,9.872827,-1.772297


In [65]:
for c in korusaq_f.columns:
    print(c)

iindex
Fractional_Day
STARTTIME
STOPTIME
UTC_obs
JDAY
INDEX
FLIGHT
LOCAL_SUN_TIME
LAT_korusaq
LON_korusaq
ALT_korusaq
PRESSURE
TEMPERATURE
THETA
O3COLUMN
SZA
WNS
WND
MSL_GPS_ALT
HAE_GPS_ALT
Radar_ALT
GRD_SPD
TAS
IAS
MachNumber
VerticalSpeed
HDG
TRK
DriftAngle
PITCH
ROLL
PotentialTemp_Hskping
Dewpoint
TotalAirTemp
IR_SurfTemp
CabinPressure
SolarZenithAngle
AircraftSunElevation
SunAzimuth
AircraftSunAzimuth
H2O_MixingRatio_Hskping
H2O_VaporPressure_Hskping
H2O_SatVaporPressureWater_Hskping
H2O_SatVaporPressureIce_Hskping
H2O_RelativeHumidity_Hskping
ProfileNumber
SiteNumber
MissedApproachNumber
H2O_MixingRatio_DLH
H2O_RelativeHumidityIce_DLH
H2O_RelativeHumidityWater_DLH
CO_korusaq
CH4_korusaq
N2O_MixingRatio
CO2_korusaq
CH2O_CAMS
C2H6_CAMS
NO2_KACES
CHOCHO_KACES
Flag_KACES
NO_MixingRatio
NOy_MixingRatio
NO2_MixingRatio
O3_korusaq
NO2_MixingRatio_LIF
PNs_MixingRatio_LIF
ANs_MixingRatio_LIF
apANs_MixingRatio_TDLIF
PAN
PPN_GTCIMS
APAN_GTCIMS
PBZN_GTCIMS
SO2_GTCIMS
HCl_GTCIMS
ClNO2_KCIMS
Cl

In [66]:
korusaq_f["vmrc2h6_GEOSChemSNU"] = korusaq_f["vmrc2h6_GEOSChemSNU"] * 1e3
korusaq_f["vmrc2h6_CAMChemNCAR"] = korusaq_f["vmrc2h6_CAMChemNCAR"] * 1e3
korusaq_f["vmrc2h6_WRFChemNCAR"] = korusaq_f["vmrc2h6_WRFChemNCAR"] * 1e3
korusaq_f["vmrc2h6_WRFChemPNU"] = korusaq_f["vmrc2h6_WRFChemPNU"] * 1e3
korusaq_f["c2h6_WRFChemUCLA"] = korusaq_f["c2h6_WRFChemUCLA"] * 1e3

In [70]:
ATS_names = ["CO", "CH4", "CH3Br", "DMS", "O3", "Ethane"]

# Create a mapping of names to specific column names for the GMI and PASTEL models
model_mapping = {
    "CO": [
        ("CO_df", "vmrco_GEOSChemSNU", "PASTEL_CO"),
        ("CO_df", "vmrco_CAMChemNCAR", "PASTEL_CO"),
        ("CO_df", "vmrco_WRFChemNCAR", "PASTEL_CO"),
        ("CO_df", "vmrco_WRFChemPNU", "PASTEL_CO"),
        ("CO_df", "co_WRFChemIOWA", "PASTEL_CO"),
        ("CO_df", "co_WRFChemUCLA", "PASTEL_CO"),
        ("CO_df", "vmrco_CMAQINHA", "PASTEL_CO"),
    ],
    "Ethane": [
        ("Ethane", "vmrc2h6_GEOSChemSNU", "PASTEL_Ethane"),
        ("Ethane", "vmrc2h6_CAMChemNCAR", "PASTEL_Ethane"),
        ("Ethane", "vmrc2h6_WRFChemNCAR", "PASTEL_Ethane"),
        ("Ethane", "vmrc2h6_WRFChemPNU", "PASTEL_Ethane"),
        ("Ethane", "c2h6_WRFChemUCLA", "PASTEL_Ethane"),
    ],
    "O3": [
        ("O3_df", "vmrco_GEOSChemSNU", "PASTEL_O3"),
        ("O3_df", "vmro3_CAMChemNCAR", "PASTEL_O3"),
        ("O3_df", "vmro3_CAMxAJOU", "PASTEL_O3"),
        ("O3_df", "vmro3_WRFChemNCAR", "PASTEL_O3"),
        ("O3_df", "vmro3_WRFChemPNU", "PASTEL_O3"),
        ("O3_df", "o3_WRFChemIOWA", "PASTEL_O3"),
        ("O3_df", "o3_WRFChemUCLA", "PASTEL_O3"),
        ("O3_df", "vmro3_CMAQINHA", "PASTEL_O3"),
    ],
    "CH4": [],
    "CH3Br": [],
    "DMS": [],
}


save_path = r"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\korusaq_micp"

# Initialize an empty list to store results
metrics_list = []

# Loop through species and calculate metrics
for species, models in model_mapping.items():
    for data_col, korusaq_col, pastel_col in models:
        # Drop NaN values from both data and target columns
        merged_korusaq = pd.concat([korusaq_f[data_col], korusaq_f[korusaq_col]], axis=1).dropna()

        merged_korusaq = merged_korusaq[merged_korusaq[korusaq_col] > 0]

        merged_pastel = pd.concat([korusaq_f[data_col], korusaq_f[pastel_col]], axis=1).dropna()

        # Ensure the same rows are kept in both DataFrames (i.e., intersection of indexes)
        merged_korusaq = merged_korusaq.loc[merged_korusaq.index.intersection(merged_pastel.index)]

        save_path = rf"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\korusaq_micp\{korusaq_col}{species}.png"
        
        scatter_plot(merged_korusaq[data_col], merged_korusaq[korusaq_col], f"{korusaq_col}_scatter", species, save_path)

        print(len(merged_korusaq))

        # Calculate metrics for GMI and PASTEL after dropping NaN
        metrics_list.append(
            calculate_and_organize_metrics_model(merged_korusaq[data_col], merged_korusaq[korusaq_col], korusaq_col, species)
        )
    save_path = rf"C:\Users\vwgei\Documents\PVOCAL\plots\model_plots\korusaq_micp\{pastel_col}{species}.png"
    scatter_plot(merged_pastel[data_col], merged_pastel[pastel_col], "pastel_sctter", species, save_path)
    metrics_list.append(
        calculate_and_organize_metrics_model(merged_pastel[data_col], merged_pastel[pastel_col], "PASTEL", species)
    )

# Combine all metrics into a single DataFrame
model_metrics = pd.concat(metrics_list, ignore_index=True)

model_metrics[['Model', 'ATS', 'RMSE', 'MAE', "R2"]]

C:\Users\vwgei\AppData\Local\Temp\ipykernel_11824\1244472152.py:71: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


538


C:\Users\vwgei\AppData\Local\Temp\ipykernel_11824\1244472152.py:71: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


493


C:\Users\vwgei\AppData\Local\Temp\ipykernel_11824\1244472152.py:71: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  ax.scatter(actual, pred, edgecolors=(0,0,0), c=color, cmap=cm)


...

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

In [68]:
print(data_col, korusaq_col, pastel_col)

O3_df vmro3_CMAQINHA PASTEL_O3


In [69]:
# #ALL of CO
# # Open the file in binary read mode and load the object
# with open(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\global_models\V4_CO\data\CO_global_X_train.pkl", 'rb') as file:
#     co_train = pickle.load(file)
# with open(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\global_models\V4_CO\data\CO_global_X_test.pkl", 'rb') as file:
#     co_test = pickle.load(file)
# co_pred_train = np.load(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\PASTEL_combined\V4_CO\CO_PASTEL_combined_pred_train.npy")
# co_pred_test = np.load(r"C:\Users\vwgei\Documents\PVOCAL\ensemble_working\PASTEL_combined\V4_CO\CO_PASTEL_combined_pred_test.npy")

# co_train['CO_PASTEL'] = co_pred_train
# co_test['CO_PASTEL'] = co_pred_test

# matching_rows = atom_gmi[atom_gmi['iindex'].isin(co_test.index)]
# matching_rows = matching_rows.set_index('iindex')

# # Find the intersection of the indexes
# matching_indexes = co_test.index.intersection(matching_rows.index)

# # Filter the DataFrames to keep only rows with the matching indexes
# var_test_matched = co_test.loc[matching_indexes]
# matching_rows_matched = matching_rows.loc[matching_indexes]

# merged = pd.concat([var_test_matched, matching_rows_matched], axis=1)

# matching_rows = df[df['iindex'].isin(merged.index)]
# matching_rows = matching_rows.set_index('iindex')

# # Find the intersection of the indexes
# matching_indexes = merged.index.intersection(matching_rows.index)

# # Filter the DataFrames to keep only rows with the matching indexes
# var_test_matched = merged.loc[matching_indexes]
# matching_rows_matched = matching_rows.loc[matching_indexes]

# final_merged = pd.concat([var_test_matched, matching_rows_matched], axis=1)

# gmi_co = calculate_and_organize_metrics_model(final_merged.iloc[:, 1], final_merged['GMI_CO_GMI'], "ATom_GMI", "CO")

# # make the final model_metrics df the first time
# model_metrics = calculate_and_organize_metrics_model(final_merged.iloc[:, 1], final_merged['CO_PASTEL'], "PASTEL_ATOM_GMI", "CO")

# model_metrics = pd.concat([model_metrics, gmi_co], ignore_index=True)

# model_metrics
# ########################################################################################


# # matching_rows = atom_cesm[atom_cesm['iindex'].isin(co_test.index)]
# # matching_rows = matching_rows.set_index('iindex')

# # # Find the intersection of the indexes
# # matching_indexes = co_test.index.intersection(matching_rows.index)

# # # Filter the DataFrames to keep only rows with the matching indexes
# # var_test_matched = co_test.loc[matching_indexes]
# # matching_rows_matched = matching_rows.loc[matching_indexes]

# # merged = pd.concat([var_test_matched, matching_rows_matched], axis=1)

# # cesm_co = calculate_and_organize_metrics_model(merged['CO'], merged['CO_CESM'], "ATom_CESM", "CO")

# # # make the final model_metrics df the first time
# # pastel_cesm_co = calculate_and_organize_metrics_model(merged['CO'], merged['CO_PASTEL'], "PASTEL_ATOM_CESM", "CO")

# # model_metrics = pd.concat([model_metrics, cesm_co, pastel_cesm_co], ignore_index=True)


# # matching_rows = seac4rs[seac4rs['iindex'].isin(co_test.index)]
# # matching_rows = matching_rows.set_index('iindex')

# # # Find the intersection of the indexes
# # matching_indexes = co_test.index.intersection(matching_rows.index)

# # # Filter the DataFrames to keep only rows with the matching indexes
# # var_test_matched = co_test.loc[matching_indexes]
# # matching_rows_matched = matching_rows.loc[matching_indexes]

# # merged = pd.concat([var_test_matched, matching_rows_matched], axis=1)

# # seac4rs_co = calculate_and_organize_metrics_model(merged['CO'], merged['CO_geos-chem'], "SEAC4RS_GEOS-Chem", "CO")

# # # make the final model_metrics df the first time
# # pastel_seac4rs_co = calculate_and_organize_metrics_model(merged['CO'], merged['CO_PASTEL'], "PASTEL_SEAC4RS", "CO")

# # model_metrics = pd.concat([model_metrics, seac4rs_co, pastel_seac4rs_co], ignore_index=True)


# # matching_rows = winter[winter['iindex'].isin(co_test.index)]
# # matching_rows = matching_rows.set_index('iindex')

# # # Find the intersection of the indexes
# # matching_indexes = co_test.index.intersection(matching_rows.index)

# # # Filter the DataFrames to keep only rows with the matching indexes
# # var_test_matched = co_test.loc[matching_indexes]
# # matching_rows_matched = matching_rows.loc[matching_indexes]

# # merged = pd.concat([var_test_matched, matching_rows_matched], axis=1)

# # winter_co = calculate_and_organize_metrics_model(merged['CO'], merged['CO_geos-chem'], "SEAC4RS_GEOS-Chem", "CO")

# # # make the final model_metrics df the first time
# # pastel_winter_co = calculate_and_organize_metrics_model(merged['CO'], merged['CO_PASTEL'], "PASTEL_SEAC4RS", "CO")

# # model_metrics = pd.concat([model_metrics, winter_co, pastel_winter_co], ignore_index=True)